# ScanConfig, SingleConfig

First we define a type **ScanConfig**.

We then create two subclasses **ExampleScanConfig** and **ExampleScanConfig2**

And create a union of these **ScanConfigsUnion** with a pydantic discriminator with the variable **type** (set within OBIBaseModel).

For each, we also define a **SingleConfig** subclass with the mixin **SingleCoordinateMixin**. This enforces that no parameter is a list.

In [ ]:
from obi_one.core.form import Form
from typing import ClassVar

from obi_one.core.block import Block
from obi_one.core.form import Form
from obi_one.core.path import NamedPath
from obi_one.core.single import SingleCoordinateMixin

"""1. Renaming two existing classes"""
class ScanConfig(Form):
    pass

class SingleConfigMixin(SingleCoordinateMixin): 
    pass


"""2. Defining two example ScanConfig classes, with associated SingleConfig classes"""

class ExampleScanConfig(ScanConfig):
    single_coord_class_name: ClassVar[str] = "ExampleSingleConfig" # Change to single_config_class_name
    name: ClassVar[str] = "Title for UI + AI Agent"
    description: ClassVar[str] = "Title for UI + AI Agent"

    class Initialize(Block):
        morphology_path: NamedPath | list[NamedPath]
        param_a: int

    initialize: Initialize

class ExampleSingleConfig(ExampleScanConfig, SingleConfigMixin):
    pass

class ExampleScanConfig2(ScanConfig):

    single_coord_class_name: ClassVar[str] = "ExampleSingleConfig2" # Change to single_config_class_name
    name: ClassVar[str] = "Single Block Generate Test"
    description: ClassVar[str] = "Test form for testing a single block form with entity SDK"

    class Initialize(Block):
        morphology_path: NamedPath | list[NamedPath]
        param_a: int

    initialize: Initialize

class ExampleSingleConfig2(ExampleScanConfig2, SingleConfigMixin):
    pass


from typing import Annotated
from pydantic import Discriminator
ScanConfigsUnion = Annotated[
    ExampleScanConfig |
    ExampleScanConfig2,
    Discriminator("type"),
]


/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "name" in "Publication" shadows an attribute in parent "Block"
  warnings.warn(
/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "name" in "ReconstructionMorphology" shadows an attribute in parent "Block"
  warnings.warn(
/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "name" in "Subject" shadows an attribute in parent "Block"
  warnings.warn(


# Define Activity for which ExampleSingleConfig is the input

In [2]:
from obi_one.core.base import OBIBaseModel
import abc

class Activity(OBIBaseModel, abc.ABC):
    pass

import entitysdk
class ExampleActivity(Activity):

    single_config: ExampleSingleConfig

    def execute(self, db_client: entitysdk.client.Client = None):

        # Arbitrary operation
        x = str(self.single_config.initialize.morphology_path) + str(self.single_config.initialize.param_a)

        if db_client:
            # Do an entitycore operation
            pass

        return x, None


# ScanGeneration, GridScanGeneration

Here we define an abstract class **ScanGeneration** Activity and a subclass **GridScanGeneration**

These convert a ScanConfig into a N SingleConfigs. Another subclass type would be a **CoupledCoordinateScanGeneration** (not shown here).

This could include the code to register the "campaign/scan" in entitycore (just the parent config and N single configs, + child entities?)

In [ ]:
import copy
import json
import logging
from collections import OrderedDict
from importlib.metadata import version
from itertools import product
from pathlib import Path

import entitysdk
from pydantic import PrivateAttr, ValidationError

from obi_one.core.block import Block
from obi_one.core.param import MultiValueScanParam, SingleValueScanParam
from obi_one.core.single import SingleCoordinateScanParams

L = logging.getLogger(__name__)


class ScanGeneration(Activity, abc.ABC):
    """Activity for creating multiple SingleConfigs where lists with multiple parameters are found"""

    scan_config: ScanConfigsUnion
    output_root: Path = Path()
    coordinate_directory_option: str = "NAME_EQUALS_VALUE"
    _multiple_value_parameters: list = None
    _coordinate_parameters: list = PrivateAttr(default=[])

    @property
    def output_root_absolute(self) -> Path:
        """Returns the absolute path of the output_root."""
        L.info(self.output_root.resolve())
        return self.output_root.resolve()

    def multiple_value_parameters(self, *, display: bool = False) -> list[MultiValueScanParam]:
        """Iterates through Blocks of self.form to find "multi value parameters".

            (i.e. parameters with list values of length greater than 1)
        - Returns a list of MultiValueScanParam objects
        """
        self._multiple_value_parameters = []

        # Iterate through all attributes of the Form
        for attr_name, attr_value in self.multi_config.__dict__.items():
            # Check if the attribute is a dictionary of Block instances
            if isinstance(attr_value, dict) and all(
                isinstance(dict_val, Block) for dict_key, dict_val in attr_value.items()
            ):
                category_name = attr_name
                category_blocks_dict = attr_value

                # If so iterate through the dictionary's Block instances
                for block_key, block in category_blocks_dict.items():
                    # Call the multiple_value_parameters method of the Block instance
                    block_multi_value_parameters = block.multiple_value_parameters(
                        category_name=category_name, block_key=block_key
                    )
                    if len(block_multi_value_parameters):
                        self._multiple_value_parameters.extend(block_multi_value_parameters)

            # Else if the attribute is a Block instance, call the _multiple_value_parameters method
            # of the Block instance
            if isinstance(attr_value, Block):
                block_name = attr_name
                block = attr_value
                block_multi_value_parameters = block.multiple_value_parameters(
                    category_name=block_name
                )
                if len(block_multi_value_parameters):
                    self._multiple_value_parameters.extend(block_multi_value_parameters)

        # Optionally display the multiple_value_parameters
        if display:
            L.info("\nMULTIPLE VALUE PARAMETERS")
            if len(self._multiple_value_parameters) == 0:
                L.info("No multiple value parameters found.")
            else:
                for multi_value in self._multiple_value_parameters:
                    L.info(f"{multi_value.location_str}: {multi_value.values}")

        # Return the multiple_value_parameters
        return self._multiple_value_parameters

    @property
    def multiple_value_parameters_dictionary(self) -> dict:
        d = {}
        for multi_value in self.multiple_value_parameters():
            d[multi_value.location_str] = multi_value.values

        return d

    def coordinate_parameters(self, *, display: bool = False) -> list[SingleCoordinateScanParams]:
        """Must be implemented by a subclass of Scan."""
        msg = "coordinate_parameters() must be implemented by a subclass of Scan."
        raise NotImplementedError(msg)

    def create_single_configs(self, *, display: bool = False) -> list[SingleConfigMixin]:
        """Coordinate instance.

        - Returns a list of "coordinate instances" by:
            - Iterating through self.coordinate_parameters()
            - Creating a single "coordinate instance" for each single coordinate parameter

        - Each "coordinate instance" is created by:
            - Making a deep copy of the form
            - Editing the multi value parameters (lists) to have the values of the single
                coordinate parameters
                (i.e. timestamps.timestamps_1.interval = [1.0, 5.0] ->
                    timestamps.timestamps_1.interval = 1.0)
            - Casting the form to its single_config_class_name type
                (i.e. SimulationsForm -> Simulation)
        """

        single_configs = []

        # Iterate through coordinate_parameters
        for idx, single_coordinate_scan_params in enumerate(self.coordinate_parameters()):
            # Make a deep copy of self.form
            single_coord_config = copy.deepcopy(self.scan_config)

            # Iterate through parameters in the single_coordinate_parameters tuple
            # Change the value of the multi parameter from a list to the single value of the
            # coordinate
            for scan_param in single_coordinate_scan_params.scan_params:
                level_0_val = single_coord_config.__dict__[scan_param.location_list[0]]

                # If the first level is a Block
                if isinstance(level_0_val, Block):
                    level_0_val.__dict__[scan_param.location_list[1]] = scan_param.value

                # If the first level is a category dictionary
                if isinstance(level_0_val, dict):
                    level_1_val = level_0_val[scan_param.location_list[1]]
                    if isinstance(level_1_val, Block):
                        level_1_val.__dict__[scan_param.location_list[2]] = scan_param.value
                    else:
                        msg = f"Non Block parameter {level_1_val} found in Form dictionary: \
                            {level_0_val}"
                        raise TypeError(msg)

            try:
                # Cast the form to its single_config_class_name type
                single_coord_config = single_coord_config.cast_to_single_coord()

                # Set the variables of the coordinate instance related to the scan
                single_coord_config.idx = idx
                single_coord_config.single_coordinate_scan_params = single_coordinate_scan_params

                # Append the coordinate instance to self._coordinate_instances
                single_configs.append(single_coord_config)

            except ValidationError as e:
                raise ValidationError(e) from e

        # Return single_configs
        return single_configs
    
    def serialize(self, output_path: Path) -> dict:
        """Serialize a Scan object.

        - type name added to each subobject of type
            inheriting from OBIBaseModel for future deserialization
        """
        # Important to use model_dump_json() instead of model_dump()
        # so OBIBaseModel's custom encoder is used to seri
        # PosixPaths as strings
        model_dump = self.model_dump_json()

        # Now load it back into an ordered dict to do some additional modifications
        model_dump = OrderedDict(json.loads(model_dump))

        # Add the obi_one version to the model_dump
        model_dump["obi_one_version"] = version("obi-one")

        # Order keys in dict
        model_dump.move_to_end("output_root", last=False)
        model_dump.move_to_end("type", last=False)
        model_dump.move_to_end("obi_one_version", last=False)

        # Order the keys in subdict "form"
        model_dump["scan_config"] = OrderedDict(model_dump["scan_config"])
        model_dump["scan_config"].move_to_end("type", last=False)

        # Create the directory and write dict to json file
        if output_path:
            with output_path.open("w", encoding="utf-8") as json_file:
                json.dump(model_dump, json_file, indent=4)

        return model_dump
    

    def execute(
        self,
        db_client: entitysdk.client.Client = None,
        ) -> entitysdk.models.core.Identifiable:

        Path.mkdir(self.output_root, parents=True, exist_ok=True)

        # Serialize the scan
        self.serialize(self.output_root / "run_scan_config.json")

        # # Create a bbp_workflow_campaign_config
        # self.create_bbp_workflow_campaign_config(
        #     self.output_root / "bbp_workflow_campaign_config.json"
        # )

        if db_client:
            pass
            # Do an entitycore operation

        single_entities = []

        single_configs = self.create_single_configs()

        # Iterate through single_configs
        for single_coord_config in single_configs:
            single_coord_config.initialize_coordinate_output_root(
                self.output_root, self.coordinate_directory_option
            )

            # Serialize the coordinate instance
            single_coord_config.serialize(
                single_coord_config.coordinate_output_root / "run_coordinate_instance.json"
            )

            if db_client:
                pass
                # Do an entitycore operation

        if db_client:
            pass
            # Do an entitycore operation


        return single_configs, None
    

class GridScanGeneration(ScanGeneration):
    """Description."""

    def coordinate_parameters(self, *, display: bool = False) -> list[SingleCoordinateScanParams]:
        """Description."""
        single_values_by_multi_value = []
        multi_value_parameters = self.multiple_value_parameters()
        if len(multi_value_parameters):
            for multi_value in multi_value_parameters:
                single_values = [
                    SingleValueScanParam(location_list=multi_value.location_list, value=value)
                    for value in multi_value.values
                ]

                single_values_by_multi_value.append(single_values)

            self._coordinate_parameters = []
            for scan_params in product(*single_values_by_multi_value):
                self._coordinate_parameters.append(
                    SingleCoordinateScanParams(scan_params=scan_params)
                )

        else:
            self._coordinate_parameters = [
                SingleCoordinateScanParams(
                    nested_coordinate_subpath_str=self.scan_config.single_coord_scan_default_subpath
                )
            ]

        # Optionally display the coordinate parameters
        if display:
            self.display_coordinate_parameters()

        # Return the coordinate parameters
        return self._coordinate_parameters


# Make an ExampleScanConfig instance and use it in a GridScanGeneration Activity

In [4]:
import obi_one as obi

initialize = ExampleScanConfig.Initialize(morphology_path=obi.NamedPath(name='test', path='test'), param_a=1)
example_scan_config = ExampleScanConfig(initialize=initialize)
grid_scan_generation = GridScanGeneration(scan_config=example_scan_config, output_root=Path("output"))
single_configs, _ = grid_scan_generation.execute()

AttributeError: 'GridScanGeneration' object has no attribute 'multi_config'

# Execute the ExampleActivity for each of the generated single configs

In [ ]:
for single_config in single_configs:
    example_activity = ExampleActivity(single_config=single_config)
    example_activity.execute()

Scan wrapper

In [ ]:
from typing import Type


class ScanWrapper(OBIBaseModel):

    scan_config: ScanConfigsUnion
    scan_generation_type: Type[ScanGeneration]
    activity_type: Type[Activity]
    output_root: Path = Path()
    coordinate_directory_option: str = "NAME_EQUALS_VALUE"

    def generate_scan(self):

        scan_generation = self.scan_generation_type(
            multi_config=self.multi_config,
            output_root=self.output_root,
            coordinate_directory_option=self.coordinate_directory_option
        )
        single_configs, _ = scan_generation.execute()

        for single_config in single_configs:
            activity = self.activity_type(single_config=single_config)
            activity.execute()

        

scan_wrapper = ScanWrapper(multi_config=example_multi_config, 
            scan_generation_type=GridScanGeneration, 
            activity_type=ExampleActivity)

scan_wrapper.generate_scan()


